In [42]:
import pandas as pd
from collections import defaultdict

def load_runlist(year_month: str) -> pd.DataFrame:
    data = pd.read_csv(
        f"/cr/data01/filip/xy-calibration/config/calib_runlists/calib_runs_{year_month}.list",
        names=["id","tel","step","date","source","mA","forDB","jobfile","comment"],
        dtype=defaultdict(lambda: str, step="int", mA="float", forDB="bool"),
        index_col=0,
        comment="#",
        sep=";",
    )

    for col in ["source", "jobfile", "comment"]:
        data[col] = data[col].map(lambda x: x.strip().replace('"',''))

    return data

def get_good_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        runlist['forDB']
        & (runlist['step'] == 6)
        & (runlist['source'] == "OLO")
        & (runlist['mA'] == 15.9)
        & (runlist['comment'] == "")
        ]

runlist=get_good_runs(load_runlist('2022-10'))

runlist

,tel,step,date,source,mA,forDB,jobfile,comment
id,,,,,,,,
15443x,co5,6,2022-10-15,OLO,15.9,True,job_Coihueco_bay5-CZ_hex_s60_r0-1300_cut.txt,
15448,co3,6,2022-10-15,OLO,15.9,True,job_Coihueco_bay3-CZ_hex_s60_r0-1300_cut.txt,
15451,co4,6,2022-10-15,OLO,15.9,True,job_Coihueco_bay4-CZ_hex_s60_r0-1300_cut.txt,
11863,la3,6,2022-10-19,OLO,15.9,True,job_LomaAmarilla_bay3-CZ_hex_s60_r0-1300_cut.txt,
11875,la4,6,2022-10-20,OLO,15.9,True,job_LomaAmarilla_bay4-CZ_hex_s60_r0-1300_cut.txt,
05616x,he1,6,2022-10-21,OLO,15.9,True,job_HEAT_bay1-CZ_hex_s60_r0-1300_cut.txt,
05621x,he2,6,2022-10-21,OLO,15.9,True,job_HEAT_bay2-CZ_hex_s60_r0-1300_cut.txt,
05626x,he3,6,2022-10-21,OLO,15.9,True,job_HEAT_bay3-CZ_hex_s60_r0-1300_cut.txt,
15672,ll4,6,2022-10-24,OLO,15.9,True,job_LosLeones_bay4-CZ_hex_s60_r0-1300_cut.txt,


In [33]:
print(runlist['source'] == "OLO")

id
15434     False
15443x    False
15448     False
15451     False
11863     False
11875     False
05616x    False
05621x    False
05626x    False
15670     False
15672     False
15675     False
Name: source, dtype: bool


In [38]:
runlist.iloc[0]['source']

'    KIT'

In [1]:
from utils.Auger.SD.Simulation import Simulation

18:35:54 (  +10.2s) [INFO   ] -- import logging
18:35:54 (  +202ms) [INFO   ] -- import numpy as np
18:35:54 (   +12ms) [INFO   ] -- import uncertainties
18:35:55 (  +482ms) [INFO   ] -- import pandas as pd
18:35:55 (    +1ms) [WARNING] -- pandas support to be dropped at some point!
18:35:55 (    +2ms) [INFO   ] -- import binaries.tools as tools
18:35:55 (    +1ms) [INFO   ] -- set MONI_PATH = '/cr/work/filip/monit_and_sd/'
18:35:55 (    +1ms) [INFO   ] -- set HIST_PATH = '/cr/work/filip/monit_and_sd/'
18:35:55 (    +0ms) [INFO   ] -- set PLOT_PATH = '/cr/data01/filip/plots/'
18:35:55 (    +1ms) [INFO   ] -- set DATA_PATH = '/cr/data01/filip/Data/'


In [2]:
sim = Simulation(name='test_sim', offline='ds_forced_station_triggers', 
                 src='/cr/users/filip/Projects/SSDTriggerEfficiency/SdSimulationReconstructionUpgrade/',
                 PRIMARY='photon')

In [3]:
# sim.run(191)

In [4]:
sim.status()

*.root files present:
  proton
    16.5_17:     1 files
  photon
    16.5_17:     1 files

*.csv files present:
  proton
    16.5_17:     1 files
  photon
    16.5_17:     0 files

{
  "PRIMARY": "photon",
  "ENERGY": "16.5_17",
  "RETHROWS": "1",
  "NPARTICLES": "30000",
  "max_idle": "150",
  "max_materialize": "150",
  "request_memory": "1G",
  "JobBatchName": "test_sim_photon_16.5_17",
  "arguments": "\"test_sim photon 16.5_17 1 30000 $(Process)\"",
  "executable": "./run.py",
  "error": "/cr/work/filip/Simulations/test_sim/log/test_sim_photon_16.5_17-$(Process).err",
  "output": "/cr/work/filip/Simulations/test_sim/log/test_sim_photon_16.5_17-$(Process).out",
  "log": "/cr/work/filip/Simulations/test_sim/log/test_sim_photon_16.5_17-$(Process).log"
}


In [5]:
sim.process()

1/1 [********************] || 00:00:00>00:00:00,    148434.02 it/s


In [6]:
sim.status()

*.root files present:
  proton
    16.5_17:     1 files
  photon
    16.5_17:     1 files

*.csv files present:
  proton
    16.5_17:     1 files
  photon
    16.5_17:     1 files

{
  "PRIMARY": "photon",
  "ENERGY": "16.5_17",
  "RETHROWS": "1",
  "NPARTICLES": "30000",
  "max_idle": "150",
  "max_materialize": "150",
  "request_memory": "1G",
  "JobBatchName": "test_sim_photon_16.5_17",
  "arguments": "\"test_sim photon 16.5_17 1 30000 $(Process)\"",
  "executable": "./run.py",
  "error": "/cr/work/filip/Simulations/test_sim/log/test_sim_photon_16.5_17-$(Process).err",
  "output": "/cr/work/filip/Simulations/test_sim/log/test_sim_photon_16.5_17-$(Process).out",
  "log": "/cr/work/filip/Simulations/test_sim/log/test_sim_photon_16.5_17-$(Process).log"
}
